In [95]:
import numpy as np
import random
import json

def sample(states_num: int = 4,
                       transition_prob: float = None,
                       noise_scale: float = 0.01,
                       seed: int = 2021):

    rng = np.random.default_rng(seed)
    if type(transition_prob)==type(None):
        transition_prob = softmax(rng.normal(size=(states_num,states_num)))
    noise = rng.normal(0,noise_scale,(states_num,states_num))
    sample_prob = transition_prob + noise
    negative_noise = sample_prob.min(axis=0)
    sample_prob = sample_prob + (negative_noise<0)*np.sign(negative_noise)*negative_noise
    sample_prob = sample_prob/sample_prob.sum(axis=0)
    return sample_prob

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)
  
#Saving matrix to a json file
data = np.ndarray.tolist(np.matrix.transpose(sample(7)))
with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

In [100]:
%%javascript
require.config({
    paths: { 
        d3: 'https://d3js.org/d3.v5.min'
    }
});
$.getJSON("data.json", function(json) {   //calling json file
    var transitionMatrix = json
    var num = transitionMatrix.length     //obtain number of states
    const alpha = Array.from(Array(num)).map((e, i) => i + 65);  
    const data = alpha.map((x) => String.fromCharCode(x));  //create state labels
    var currentState = 0;
    var pi = Math.PI //determine Pi
    var m = data.length; // number of states

    (function(element) {
        require(['d3'], function(d3) {
            var svg = d3.select(element.get(0))  //set up SVG
                        .append('svg')
                        .attr('width', 750)
                        .attr('height', 750);

            function circleTransition(){
                var filter = svg.append("defs")  //blur filter for glow effect
                    .append("filter")
                    .attr("id", "blur")
                    .append("feGaussianBlur")
                    .attr("stdDeviation", 2); 
                
                var timeCircle = svg.append("circle")
                    .attr("r", 40);
                
                var timeGlowCurrent = svg.append("circle") //current state indicator
                    .attr("filter", "url(#blur)");   
                
                var timeGlowNext = svg.append("circle")  //next state indicator
                    .attr("filter", "url(#blur)");
                
                var textCurrentState = svg.append("text")
                    .attr("font-size",0)
                    .attr("font-weight",500)
                    .attr('text-anchor','middle')
                
                var textNextState = svg.append("text")
                    .attr("font-size",0)
                    .attr("font-weight",500)
                    .attr('text-anchor','middle')
                
                var textTO = svg.append("text")
                    .attr('x',650)
                    .attr('y',200)
                    .attr('font-size', 20)
                    .attr("font-weight",500)
                    .attr('text-anchor','middle')
                    .text("TO")
                
                var title = svg.append("text")
                    .attr("font-weight",500)
                    .attr('x',100)
                    .attr('y',50)
                    .attr('font-size', 30)
                    .text("Markov Chain Dynamic Visualisation")

                
                var g = svg.selectAll('.someClass')
                    .data(data)
                    .enter()
                    .append("g")
                    .attr("transform", function(d,i) {
                      return "translate(" + (340+ 200*Math.cos(i*2*pi/m)) + "," + (200*Math.sin(i*2*pi/m)+360) + ")";
                    });  //to position states in a circle
        
                    g.append("circle")
                      .attr("cx", 0)
                      .attr("cy", 0)
                      .attr("r",30)
                      .style("fill", function(d,i){
                var num = (i+1)*255/m
                var num2 = 255 - (i+1)*255/m
                    return "rgb("+num+",200,"+num2+")"});
        
                    g.append("text")
                      .style("fill", "white")
                      .attr("x",0)
                      .attr("y",7)
                      .attr("text-anchor","middle")
                      .attr("font-size",20)
                      .attr("font-weight",500)
                      .text(function(d) {
                        return d;
                      })
                repeat();
                function repeat() {
                    var i = currentState;
                    var nextStates = transitionMatrix[i];
                    var nextState = -1;
                    var rand = Math.random();
                    var total = 0;
                    for(var j = 0; j < nextStates.length; j++) {
                        total += nextStates[j];  // generating next state
                    if(rand < total) {
                        nextState = j;
                        break;
                        }
                    }
                    var cx_current =340+200*Math.cos(currentState*2*pi/m)
                    var cy_current =360+200*Math.sin(currentState*2*pi/m)
                    var cx_next =340+200*Math.cos(nextState*2*pi/m)
                    var cy_next = 360+200*Math.sin(nextState*2*pi/m)
                  timeCircle  //animations
                    .style("opacity",0.5)
                    .attr("fill", "yellow")
                    .attr('cx', cx_current)
                    .attr('cy',cy_current)
                    .transition()
                    .ease(d3.easePoly)
                    .delay(200)
                    .duration(800)
                    .attr('r',15)
                    .attr("fill", "orange")
                    .style("opacity",1.0)
                    .attr('cx', 340)
                    .attr('cy',360)
                    .transition()
                    .duration(1000)
                    .attr("fill", "yellow")
                    .attr('r',40)
                    .style("opacity",0.5)
                    .attr('cx', cx_next)
                    .attr('cy', cy_next)
                    .on("end", repeat);
              
                  timeGlowCurrent
                    .style("opacity",0)
                    .attr("fill","white")
                    .attr('cx', cx_current)
                    .attr('cy',cy_current)
                    .attr('r',40)
                    .transition()
                    .ease(d3.easeLinear)
                    .duration(200)
                    .attr("fill", "yellow")
                    .style("opacity",0.3)
                    .attr('r',45)
                    .transition()
                    .delay(1600)
                    .duration(200)
                    .attr("fill", "white")
                    .style("opacity",0)
                    .attr('r',60)
              
                  timeGlowNext
                    .style("opacity",0)
                    .attr("fill","white")
                    .attr('cx', cx_next)
                    .attr('cy',cy_next)
                    .attr('r',40)
                    .transition()
                    .ease(d3.easeLinear)
                    .duration(200)
                    .attr("fill", "pink")
                    .style("opacity",0.3)
                    .attr('r',45)
                    .transition()
                    .delay(1600)
                    .duration(200)
                    .attr("fill", "white")
                    .style("opacity",0)
                    .attr('r',60)
                    
                  textCurrentState
                    .attr('x',600)
                    .attr('y',200)
                    .attr('fill', function(d){
                        var num = (currentState+1)*255/m
                        var num2 = 255 - (currentState+1)*255/m
                        return "rgb("+num+",200,"+num2+")"})
                    .text(data[currentState])
                    .transition()
                    .duration(800)
                    .attr('font-size',40)
                    .transition()
                    .duration(800)
                    .delay(400)
                    .attr('font-size',0)
                    
                  textNextState
                    .attr('x',700)
                    .attr('y',200)
                    .attr('fill', function(d){
                        var num = (nextState+1)*255/m
                        var num2 = 255 - (nextState+1)*255/m
                        return "rgb("+num+",200,"+num2+")"})
                    .text(data[nextState])
                    .transition()
                    .duration(800)
                    .attr('font-size',40)
                    .transition()
                    .delay(400)
                    .duration(800)
                    .attr('font-size',0)
                
                  textTO
                    .transition()
                    .duration(800)
                    .attr('fill','rgb(100,150,150)')
                    .attr('font-size',30)
                    .transition()
                    .duration(800)
                    .delay(400)
                    .attr('font-size',20)
                    .attr('fill','rgb(150,200,200)')
                    
                  title
                    .attr('fill','rgb(50,150,150)')
                    .transition()
                    .duration(1000)
                    .attr('fill','rgb(50,150,150)')
                    .transition(1000)
                    .attr('fill','rgb(50,100,150)')
                  currentState = nextState;
                };
            };
        circleTransition()})
    })(element);
});

<IPython.core.display.Javascript object>